In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
! ls ../data

UCI_Credit_Card.csv UCI_Credit_Card.zip


In [43]:
# data = pd.read_csv("../data/UCI_Credit_Card.csv")
data = pd.read_csv("../data/creditcard.csv")

In [44]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [46]:
data['Class'].value_counts(normalize=True)

0    0.998273
1    0.001727
Name: Class, dtype: float64

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['default.payment.next.month','ID'],axis=1),
    data['default.payment.next.month'] ,
    test_size=0.33, 
    random_state=42
)

In [17]:
from bokbokbok.loss_functions.classification import WeightedCrossEntropyLoss
from bokbokbok.eval_metrics.classification import WeightedCrossEntropyMetric

from sklearn.metrics import roc_auc_score, f1_score, precision_recall_curve, auc, roc_curve


In [42]:
import lightgbm as lgb


train = lgb.Dataset(X_train, y_train)
test = lgb.Dataset(X_test, y_test, reference=train)
params = {
    'num_leaves': 10,
    'n_estimators':100, 
#     'class_weight' : "balanced",
     'n_jobs': 2,
     'learning_rate': 0.1,
    'metric' : [
       # 'auc', 
        'binary_logloss'
    ],
    "verbose":0
   }

clf = lgb.train(params=params,
                train_set=train,
                valid_sets=[train, test],
                valid_names=['train','test'],
#                 fobj=WeightedFocalLoss(alpha=alpha, gamma=gamma),
#                 feval=WeightedFocalMetric(alpha=alpha, gamma=gamma),
                early_stopping_rounds=10,
                verbose_eval=15
               )

# roc_auc_score(y_test, clip_sigmoid(clf.predict(X_valid)))

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001083 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 10 rounds
[15]	train's binary_logloss: 0.434612	test's binary_logloss: 0.435991
[30]	train's binary_logloss: 0.422605	test's binary_logloss: 0.428331
[45]	train's binary_logloss: 0.416895	test's binary_logloss: 0.426518
[60]	train's binary_logloss: 0.412939	test's binary_logloss: 0.426046
[75]	train's binary_logloss: 0.409236	test's binary_logloss: 0.42573
Early stopping, best iteration is:
[65]	train's binary_logloss: 0.411566	test's binary_logloss: 0.42562


In [ ]:
params = {
    'num_leaves': 10,
    'n_estimators':100, 
   'class_weight' : "balanced",
     'n_jobs': 2,
     'learning_rate': 0.1,
    'metric' : [
       # 'auc', 
        'binary_logloss'
    ],
    "verbose":0
   }

clf = lgb.train(params=params,
                train_set=train,
                valid_sets=[train, test],
                valid_names=['train','test'],
#                 fobj=WeightedFocalLoss(alpha=alpha, gamma=gamma),
#                 feval=WeightedFocalMetric(alpha=alpha, gamma=gamma),
                early_stopping_rounds=10,
                verbose_eval=15
               )



In [31]:
# clf.predict(X_test)
roc_auc_score(y_test, clf.predict(X_test))
# roc_auc_score(y_train, clf.predict(X_train))

0.781653290825604

In [26]:
y_train.value_counts(normalize=True)

0    0.777214
1    0.222786
Name: default.payment.next.month, dtype: float64

In [ ]:
import lightgbm as lgb

In [ ]:

# precision, recall, thresholds = precision_recall_curve(y_test, clf.predict_proba(X_test)[:,1])
# area = auc(recall, precision)

def fit_model(model, *, X_train, y_train, X_test, y_test):

    eval_set = [(X_train, y_train), (X_test, y_test)]
    model.fit(
        X_train, 
        y_train, 
        eval_metric=["logloss"], 
        eval_set=eval_set, 
        verbose=0,
    early_stopping_rounds=10)
    
    
    return model


def _eval(model, X, y, thr = 0.5):
    
    proba_0 = model.predict_proba(X)[:,0]
    proba_1 = model.predict_proba(X)[:,1]
    
    auc = round(roc_auc_score(y, proba_1),4)
    
    out = {
        "auc": auc
    }
    
    return out
    

def eval_model(model, *, X_train=None, y_train=None, X_test=None, y_test=None,thr = 0.5):
    
    if X_train is not None:
        train_res = _eval( model, X_train, y_train, thr=thr)
        print("\nTrain")
        print(train_res)
    
    if X_test is not None:
        test_res = _eval( model, X_test, y_test, thr=thr)
        print("\nTest")
        print(test_res)
    

    

def plot_logloss(model, ax = None, **plot_kwargs):
    
    results = model.evals_result_

    epochs = len(results['training']['binary_logloss'])
    x_axis = range(0, epochs)
    # plot log loss
    if ax is None:
        fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(x_axis, results['training']['binary_logloss'], label='Train', **plot_kwargs)
    ax.plot(x_axis, results['valid_1']['binary_logloss'], label='Test', **plot_kwargs)
    ax.legend()
    # make predictions for test data
    
    


### Default model

In [ ]:
def_model = lgb.LGBMClassifier(num_leaves = 10, n_estimators=100)

def_model = fit_model(def_model, X_train = X_train, y_train = y_train, X_test = X_test, y_test = y_test)
# plot_logloss(def_model)
eval_model(def_model, X_train = X_train, y_train = y_train, X_test = X_test, y_test = y_test)

In [ ]:
rescaled_model = lgb.LGBMClassifier(num_leaves = 10, n_estimators=100, class_weight = "balanced")
# rescaled_model = lgb.LGBMClassifier(num_leaves = 4, n_estimators=20)

rescaled_model = fit_model(rescaled_model, X_train = X_train, y_train = y_train, X_test = X_test, y_test = y_test)
# plot_logloss(rescaled_model)
eval_model(rescaled_model, X_train = X_train, y_train = y_train, X_test = X_test, y_test = y_test)

# Check ordering

In [ ]:
proba_def=np.round(def_model.predict_proba(X_test)[:,1],3)
proba_rescaled=np.round(rescaled_model.predict_proba(X_test, )[:,1],3)

proba_def_raw=def_model.predict_proba(X_test, raw_score=True)
proba_rescaled_raw=rescaled_model.predict_proba(X_test, raw_score=True)

df_scores = pd.DataFrame(
    {
      'proba_default':proba_def,
      'proba_rescaled':proba_rescaled,
      'log_odds_default':proba_def_raw,
      'log_odds_rescaled':proba_rescaled_raw,
      'y':y_test
        
    }
)

In [ ]:
# pd.qcut?

In [ ]:
def agg_buckets(df, col, q):
    df_stats = (
        df.sort_values(by=col)
        .groupby(pd.qcut(df[col], q=q, duplicates='drop'), as_index = False)
        .agg(
            tot_def = pd.NamedAgg("y","sum"),
            tot_entries = pd.NamedAgg("y","count"),
            def_rate = pd.NamedAgg("y","mean"),
            mean_proba = pd.NamedAgg("proba_default","mean"),
            mean_proba_resc = pd.NamedAgg("proba_rescaled","mean")
        )
    )
    df_stats['tot_goods'] = df_stats['tot_entries'] - df_stats['tot_def']
    
    df_stats['pc_goods'] = df_stats['tot_goods']/df_stats['tot_goods'].sum()
    df_stats['pc_def'] = df_stats['tot_def']/df_stats['tot_def'].sum()
    
    df_stats['woe'] = np.log(df_stats['pc_goods']/df_stats['pc_def'])
    df_stats['iv'] = (df_stats['pc_goods']-df_stats['pc_def'])* df_stats['woe']
    
    columns = [
        'tot_entries','tot_goods','tot_def',
        'def_rate','mean_proba','mean_proba_resc',
        'woe','iv'
    ]
#     return df_stats
    return df_stats[columns]

def_stats = agg_buckets(df_scores, col = 'log_odds_default',q=8)

resc_stats = agg_buckets(df_scores, col = 'log_odds_rescaled',q=8)

In [ ]:
display(def_stats)
display(resc_stats)

In [ ]:
def_stats['iv'].sum(), resc_stats['iv'].sum()

In [ ]:
fig, ax = plt.subplots()
ax.plot(def_stats['woe'], resc_stats['woe'])
ax.plot([def_stats['woe'].min(), def_stats['woe'].max()], [resc_stats['woe'].min(), resc_stats['woe'].max()], linestyle='--', lw=2, color='r',
        label='Chance', alpha=.8)
ax.grid()

In [ ]:
fig, ax = plt.subplots()
ax.plot(proba_def_raw)
ax.plot(proba_def_raw)

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.gradient(proba_def_raw))
ax.plot(np.gradient(proba_rescaled_raw), alpha=0.6)

In [ ]:
fig, ax = plt.subplots()
ax.plot(ratio_probas.clip(-10,10))

In [ ]:
fig, ax = plt.subplots()
plot_logloss(def_model, ax = ax, linestyle = "--")
plot_logloss(rescaled_model, ax = ax)

In [ ]:
# fig, ax = plt.subplots(2,1,figsize=(12,10))
# lgb.plot_tree(def_model, ax = ax[0])
# lgb.plot_tree(rescaled_model, ax = ax[1])

### Plot roc curve

In [ ]:
roc_def = roc_curve(y_test,def_model.predict_proba(X_test)[:,1])
roc_rescaled = roc_curve(y_test,rescaled_model.predict_proba(X_test)[:,1])

pr_def_1 = precision_recall_curve(y_test,def_model.predict_proba(X_test)[:,1])
pr_def_0 = precision_recall_curve(y_test,def_model.predict_proba(X_test)[:,0],pos_label=0)

pr_rescaled_1 = precision_recall_curve(y_test,rescaled_model.predict_proba(X_test)[:,1])
pr_rescaled_0 = precision_recall_curve(y_test,rescaled_model.predict_proba(X_test)[:,0],pos_label=0)

In [ ]:


fig, ax = plt.subplots(1,2,figsize=(16,8))
ax[0].plot(pr_def_1[1], pr_def_1[0], color='blue',
        label="Standard model",
        lw=2, alpha=.8)
ax[0].plot(pr_rescaled_1[1], pr_rescaled_1[0], color='red',
        label="Rescaled model",
        lw=2, alpha=.8)
ax[0].set_ylabel("Precision")
ax[0].set_xlabel("Recall")

ax[0].set_title("Class 1")
ax[0].set_ylim(0,1)
ax[0].set_xlim(0,1)
ax[0].grid()
ax[0].legend(loc = 'lower left')

ax[1].plot(pr_def_0[1], pr_def_0[0], color='blue',
        label="Standard model",
        lw=2, alpha=.8)
ax[1].plot(pr_rescaled_0[1], pr_rescaled_0[0], color='red',
        label="Rescaled model",
        lw=2, alpha=.8)
ax[1].set_ylabel("Precision")
ax[1].set_xlabel("Recall")

ax[1].set_title("Class 0")

ax[1].set_ylim(0,1)
ax[1].set_xlim(0,1)
ax[1].grid()
ax[1].legend(loc = 'lower left')


# ax.plot(roc_rescaled[0], roc_rescaled[1], color='Green',
#         label=" Rescaled Standard_model",
#         lw=2, alpha=.8)

In [ ]:
pr_rescaled_0

In [ ]:
ax[0].legend

In [ ]:
precision_recall_curve.

In [ ]:
roc_def[2].mean()

In [ ]:
roc_rescaled[2].mean()

In [ ]:
pd.qcut(roc_def[2], q=10)

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
ax.plot(roc_def[0], roc_def[1], color='blue',
        label="Standard model",
        lw=2, alpha=.8)


ax.plot(roc_rescaled[0], roc_rescaled[1], color='Green',
        label=" Rescaled Standard_model",
        lw=2, alpha=.8)

ax2 = ax.twinx()
ax2.plot(roc_def[0], roc_def[2], color='lightblue',
        label="Standard model model thresholds",
        lw=2, alpha=.8)
ax2.plot(roc_rescaled[0], roc_rescaled[2], color='lightgreen',
        label="Rescaled model model thresholds",
        lw=2, alpha=.8)

ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Chance', alpha=.8)

ax.legend()

In [ ]:
def_model.predict_proba(X_test)

In [ ]:
X_test.shape

In [ ]:
results['training'].keys()

In [ ]:
results = model.evals_result_


epochs = len(results['training']['binary_logloss'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = plt.subplots(figsize=(10,8))
ax.plot(x_axis, results['training']['binary_logloss'], label='Train')
ax.plot(x_axis, results['valid_1']['binary_logloss'], label='Test')
ax.legend()

In [ ]:
np.unique(model.apply(X_train)[:,-1])

In [ ]:

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
# retrieve performance metrics
results = model.evals_result()
epochs = len(results['validation_0']['error'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
ax.legend()
pyplot.ylabel('Log Loss')
pyplot.title('XGBoost Log Loss')
pyplot.show()
# plot classification error
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['error'], label='Train')
ax.plot(x_axis, results['validation_1']['error'], label='Test')
ax.legend()
pyplot.ylabel('Classification Error')
pyplot.title('XGBoost Classification Error')
pyplot.show()